In [1]:
# 6. Triển khai Support Vector Machine (SVM)

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import sys
import os
import joblib 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path:
    sys.path.append(module_path)

try:
    from src.preprocess import clean_text_v1
except ImportError:
    print("Lỗi: Không thể import hàm clean_text_v1 từ src/preprocess.py.")
    print("Hãy đảm bảo file tồn tại và chứa hàm cần thiết.")
    exit()
except Exception as e:
    print(f"Lỗi khác khi import hàm tiền xử lý: {e}")
    exit()

print("Đang đọc dữ liệu...")
try:
    data_path = '../data/processed/spam_cleaned_columns.csv'
    df = pd.read_csv(data_path, encoding='latin-1')
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file {data_path}")
    exit()



print("Đang áp dụng tiền xử lý văn bản...")
df['Cleaned_Message'] = df['Message'].apply(clean_text_v1)
print("Tiền xử lý văn bản hoàn tất.")
print("5 dòng đầu với Cleaned_Message:")
print(df[['Message', 'Cleaned_Message', 'Category']].head())

X = df['Cleaned_Message']
y = df['Category_Num'] # 0: ham, 1: spam



Đang đọc dữ liệu...
Đang áp dụng tiền xử lý văn bản...
Tiền xử lý văn bản hoàn tất.
5 dòng đầu với Cleaned_Message:
                                             Message  \
0  Go until jurong point, crazy.. Available only ...   
1                      Ok lar... Joking wif u oni...   
2  Free entry in 2 a wkly comp to win FA Cup fina...   
3  U dun say so early hor... U c already then say...   
4  Nah I don't think he goes to usf, he lives aro...   

                                     Cleaned_Message Category  
0  go jurong point crazy available bugis n great ...      ham  
1                            ok lar joking wif u oni      ham  
2  free entry 2 wkly comp win fa cup final tkts 2...     spam  
3                u dun say early hor u c already say      ham  
4        nah dont think goes usf lives around though      ham  


In [8]:
from imblearn.over_sampling import SMOTE # Import SMOTE
from sklearn.model_selection import cross_val_predict

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
#stratify: chia dữ liệu theo tỷ lệ của nhãn
#random_state: đảm bảo kết quả chia dữ liệu giống nhau
print(f"\nĐã chia dữ liệu: {X_train.shape[0]} huấn luyện, {X_test.shape[0]} kiểm tra")


print("Đang thực hiện vector hóa TF-IDF...")
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train) 



# print("Đang lấy danh sách tên đặc trưng...")
# try:
#     feature_names = tfidf_vectorizer.get_feature_names_out()
#     print(f"Số lượng đặc trưng: {len(feature_names)}")
    
#     print("\nKiểm tra các n-grams quan trọng:")
#     important_ngrams = ["limited time", "time offer", "exclusive discount", 
#                           "50 percent", "percent discount", "this friday", "friday only",
#                           "limited", "offer", "exclusive", "discount", "percent", "friday", "only"] # Thêm cả unigrams
    
#     found_ngrams = []
#     for ngram in important_ngrams:
#         if ngram in feature_names:
#             found_ngrams.append(ngram)
#             print(f"'{ngram}' CÓ trong từ điển.")
#         else:
#             print(f"'{ngram}' KHÔNG CÓ trong từ điển.")
    
#     if not found_ngrams:
#         print("Không tìm thấy n-gram quan trọng nào trong từ điển với max_features=5000 và ngram_range=(1,2).")
#         print("Hãy thử tăng max_features hoặc sử dụng min_df thay thế.")

# except Exception as e:
#     print(f"Lỗi khi lấy tên đặc trưng: {e}")


X_test_tfidf = tfidf_vectorizer.transform(X_test)


print("\nPhân bố lớp trước khi áp dụng SMOTE trên tập huấn luyện:")
print(y_train.value_counts())

print("Đang áp dụng SMOTE trên tập huấn luyện...")
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_tfidf, y_train)

print("\nPhân bố lớp sau khi áp dụng SMOTE trên tập huấn luyện:")
print(pd.Series(y_train_smote).value_counts()) # Chuyển y_train_smote sang Series để dùng value_counts()
print(f"Kích thước X_train_smote: {X_train_smote.shape}")



print(f"Kích thước ma trận TF-IDF tập huấn luyện: {X_train_tfidf.shape}")
print(f"Kích thước ma trận TF-IDF tập kiểm tra: {X_test_tfidf.shape}")

print("\nBắt đầu huấn luyện mô hình SVM...")

model_svm_smote = SVC(kernel='linear', class_weight='balanced', probability=True, random_state=42)

model_svm_smote.fit(X_train_smote, y_train_smote)
print("Đã huấn luyện xong mô hình SVM.")

   

print("\nĐánh giá mô hình trên tập kiểm thử (sau SMOTE):")
    # Nếu dùng GridSearchCV, sử dụng best_svm_smote
    # y_pred_smote = best_svm_smote.predict(X_test_tfidf) 
    
    # Nếu huấn luyện trực tiếp
y_pred_smote = model_svm_smote.predict(X_test_tfidf)

print("Confusion Matrix (sau SMOTE):")
print(confusion_matrix(y_test, y_pred_smote))
print("\nClassification Report (sau SMOTE):")
print(classification_report(y_test, y_pred_smote, target_names=['HAM (0)', 'SPAM (1)']))




Đã chia dữ liệu: 4457 huấn luyện, 1115 kiểm tra
Đang thực hiện vector hóa TF-IDF...
Đang lấy danh sách tên đặc trưng...
Số lượng đặc trưng: 5000

Kiểm tra các n-grams quan trọng:
'limited time' KHÔNG CÓ trong từ điển.
'time offer' KHÔNG CÓ trong từ điển.
'exclusive discount' KHÔNG CÓ trong từ điển.
'50 percent' KHÔNG CÓ trong từ điển.
'percent discount' KHÔNG CÓ trong từ điển.
'this friday' KHÔNG CÓ trong từ điển.
'friday only' KHÔNG CÓ trong từ điển.
'limited' KHÔNG CÓ trong từ điển.
'offer' CÓ trong từ điển.
'exclusive' KHÔNG CÓ trong từ điển.
'discount' CÓ trong từ điển.
'percent' KHÔNG CÓ trong từ điển.
'friday' CÓ trong từ điển.
'only' KHÔNG CÓ trong từ điển.

Phân bố lớp trước khi áp dụng SMOTE trên tập huấn luyện:
Category_Num
0    3859
1     598
Name: count, dtype: int64
Đang áp dụng SMOTE trên tập huấn luyện...

Phân bố lớp sau khi áp dụng SMOTE trên tập huấn luyện:
Category_Num
0    3859
1    3859
Name: count, dtype: int64
Kích thước X_train_smote: (7718, 5000)
Kích thước ma

In [22]:
#save model
model_path = '../results/trained_models/svm_smote_model.pkl'
joblib.dump(model_svm_smote, model_path)

['../results/trained_models/svm_smote_model.pkl']

In [12]:
y_train_pred_smote = cross_val_predict(model_svm_smote, X_train_smote, y_train_smote, cv=5)
print("\nĐánh giá mô hình trên tập huấn luyện (sau SMOTE):")
print("Confusion Matrix (tập huấn luyện):")
print(confusion_matrix(y_train_smote, y_train_pred_smote))
print(classification_report(y_train_smote, y_train_pred_smote, target_names=['HAM (0)', 'SPAM (1)']))


Đánh giá mô hình trên tập huấn luyện (sau SMOTE):
Confusion Matrix (tập huấn luyện):
[[3822   37]
 [ 152 3707]]
              precision    recall  f1-score   support

     HAM (0)       0.96      0.99      0.98      3859
    SPAM (1)       0.99      0.96      0.98      3859

    accuracy                           0.98      7718
   macro avg       0.98      0.98      0.98      7718
weighted avg       0.98      0.98      0.98      7718



In [13]:
y_test_pred = cross_val_predict(model_svm_smote, X_test_tfidf, y_test, cv=5)
print("\nĐánh giá mô hình trên tập kiểm thử (sau SMOTE):")
print("Confusion Matrix (tập kiểm thử):")
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred, target_names=['HAM (0)', 'SPAM (1)']))


Đánh giá mô hình trên tập kiểm thử (sau SMOTE):
Confusion Matrix (tập kiểm thử):
[[959   7]
 [ 43 106]]
              precision    recall  f1-score   support

     HAM (0)       0.96      0.99      0.97       966
    SPAM (1)       0.94      0.71      0.81       149

    accuracy                           0.96      1115
   macro avg       0.95      0.85      0.89      1115
weighted avg       0.95      0.96      0.95      1115



In [10]:
from sklearn.metrics import f1_score, precision_score, recall_score
print("\nF1 Score (tập huấn luyện):", f1_score(y_train_smote, y_train_pred_smote))
print("Precision (tập huấn luyện):", precision_score(y_train_smote, y_train_pred_smote))
print("Recall (tập huấn luyện):", recall_score(y_train_smote, y_train_pred_smote))


F1 Score (tập huấn luyện): 0.9751413915559648
Precision (tập huấn luyện): 0.9901175213675214
Recall (tập huấn luyện): 0.9606115573982897


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print("\nBắt đầu huấn luyện mô hình SVM không SMOTE...")
model_svm = SVC(kernel='linear', class_weight='balanced', probability=True, random_state=42, C=0.1)
y_train_pred = cross_val_predict(model_svm, X_train_tfidf, y_train, cv=5)
print(confusion_matrix(y_train, y_train_pred))
print(classification_report(y_train, y_train_pred, target_names=['HAM (0)', 'SPAM (1)']))
from sklearn.metrics import f1_score, precision_score, recall_score
print("\nF1 Score (tập huấn luyện):", f1_score(y_train, y_train_pred))
print("Precision (tập huấn luyện):", precision_score(y_train, y_train_pred))
print("Recall (tập huấn luyện):", recall_score(y_train, y_train_pred))


Bắt đầu huấn luyện mô hình SVM không SMOTE...
[[3804   55]
 [  70  528]]
              precision    recall  f1-score   support

     HAM (0)       0.98      0.99      0.98      3859
    SPAM (1)       0.91      0.88      0.89       598

    accuracy                           0.97      4457
   macro avg       0.94      0.93      0.94      4457
weighted avg       0.97      0.97      0.97      4457


F1 Score (tập huấn luyện): 0.8941574936494496
Precision (tập huấn luyện): 0.9056603773584906
Recall (tập huấn luyện): 0.882943143812709


In [20]:
y_test_pred = cross_val_predict(model_svm, X_test_tfidf, y_test, cv=5)
print("\nĐánh giá mô hình trên tập kiểm thử (sau SMOTE):")
print("Confusion Matrix (tập kiểm thử):")
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred, target_names=['HAM (0)', 'SPAM (1)']))


Đánh giá mô hình trên tập kiểm thử (sau SMOTE):
Confusion Matrix (tập kiểm thử):
[[960   6]
 [ 42 107]]
              precision    recall  f1-score   support

     HAM (0)       0.96      0.99      0.98       966
    SPAM (1)       0.95      0.72      0.82       149

    accuracy                           0.96      1115
   macro avg       0.95      0.86      0.90      1115
weighted avg       0.96      0.96      0.95      1115



In [4]:

new_messages_for_test = [
    "URGENT! Claim your FREE £1000 prize now! Click http://spamlink.com", # Tin nhắn có vẻ là SPAM
    "Hey, wondering if you're free for coffee later today?", # Tin nhắn có vẻ là HAM
    "Meet singles in your area, text DATE to 88888 T&Cs apply 18+", # Tin nhắn có vẻ là SPAM
    "Remember to buy bread and eggs on your way back.", # Tin nhắn có vẻ là HAM
    "Limited time offer! Exclusive discount just for you!", # Tin nhắn có vẻ là SPAM
    "50 percent discount this Friday only!" #SPAM
]

print("--- Bắt đầu Test Case ---")
print(f"Số lượng tin nhắn thử nghiệm: {len(new_messages_for_test)}")

try:
    cleaned_test_messages = [clean_text_v1(msg) for msg in new_messages_for_test]
    print("\nTin nhắn sau khi tiền xử lý:")
    for i, msg in enumerate(cleaned_test_messages):
        print(f"{i+1}: {msg}")
except NameError:
    print("\nLỗi: Hàm 'clean_text_v1' chưa được định nghĩa hoặc import.")
    print("Vui lòng chạy cell định nghĩa/import hàm này trước.")
    raise


try:
    new_messages_tfidf_test = tfidf_vectorizer.transform(cleaned_test_messages)
    print(f"\nĐã vector hóa {len(cleaned_test_messages)} tin nhắn thành ma trận TF-IDF kích thước: {new_messages_tfidf_test.shape}")
except NameError:
    print("\nLỗi: Biến 'tfidf_vectorizer' chưa được định nghĩa.")
    print("Vui lòng chạy cell huấn luyện TF-IDF Vectorizer trước.")
    raise

print("\nĐang thực hiện dự đoán bằng mô hình SVM...")
try:
    predictions_test = model_svm_smote.predict(new_messages_tfidf_test)
    
    probabilities_test = model_svm_smote.predict_proba(new_messages_tfidf_test)
except NameError:
    print("\nLỗi: Biến 'svm_model' chưa được định nghĩa.")
    print("Vui lòng chạy cell huấn luyện mô hình SVM trước.")
    raise
except AttributeError:
     print("\nLỗi: Không thể lấy xác suất.")
     print("Để dùng predict_proba(), mô hình SVM cần được khởi tạo với tham số 'probability=True'.")
     probabilities_test = np.array([[0.5, 0.5]] * len(predictions_test)) 

# --- 4. Hiển thị kết quả ---
print("\n--- Kết quả Dự đoán Test Case ---")
label_map = {0: 'ham', 1: 'spam'} 
for i, original_message in enumerate(new_messages_for_test):
    predicted_label_num = predictions_test[i]
    predicted_label_text = label_map[predicted_label_num]
    prob_ham = probabilities_test[i][0]
    prob_spam = probabilities_test[i][1] 

    print(f"\nTin nhắn gốc : \"{original_message}\"")
    print(f"  -> Dự đoán   : {predicted_label_text.upper()}")
    if 'AttributeError' not in locals() or not isinstance(locals()['AttributeError'], AttributeError):
         print(f"  -> Xác suất : [Ham={prob_ham:.4f}, Spam={prob_spam:.4f}]")

print("\n--- Hoàn thành Test Case ---")

--- Bắt đầu Test Case ---
Số lượng tin nhắn thử nghiệm: 6

Tin nhắn sau khi tiền xử lý:
1: urgent claim free 1000 prize click httpspamlinkcom
2: hey wondering youre free coffee later today
3: meet singles area text date 88888 tcs apply 18
4: remember buy bread eggs way back
5: limited time offer exclusive discount
6: 50 percent discount friday

Đã vector hóa 6 tin nhắn thành ma trận TF-IDF kích thước: (6, 5000)

Đang thực hiện dự đoán bằng mô hình SVM...

--- Kết quả Dự đoán Test Case ---

Tin nhắn gốc : "URGENT! Claim your FREE £1000 prize now! Click http://spamlink.com"
  -> Dự đoán   : SPAM
  -> Xác suất : [Ham=0.0000, Spam=1.0000]

Tin nhắn gốc : "Hey, wondering if you're free for coffee later today?"
  -> Dự đoán   : HAM
  -> Xác suất : [Ham=0.9997, Spam=0.0003]

Tin nhắn gốc : "Meet singles in your area, text DATE to 88888 T&Cs apply 18+"
  -> Dự đoán   : SPAM
  -> Xác suất : [Ham=0.0000, Spam=1.0000]

Tin nhắn gốc : "Remember to buy bread and eggs on your way back."
  -> Dự đoán

B1:Tạo 1 file google doc -> Note lại toàn bộ kết quả baseline của từng thuật toán (accuracy, precision, recall, f1-score, confusion matrix)
B2: Tinh chỉnh siêu tham số
- random forest
- Logistic Regression
B3: Sử dụng GridSearchCV hoặc RandomizedSearchCV (CV: Cross validation)
- Cross Validation: K-Fold Cross Validation (Kiểm định chéo K lần)
  + Chia dữ liệu: Chia ngẫu nhiên tập train thành K phần, giá trị K thường là 5 hoặc 10
  + Lặp K lần: Quá trình huấn luyện và đánh giá dược lặp lại K lần. Trong mỗi lần lặp (ví dụ lần lặp thứ i):
    > Chọn Fold kiểm tra: Fold thứ i được giữ lại làm tập dữ liệu kiểm định (validation set) hoặc là "test fold" cho lần lặp này.
    >Chọn fold huấn luyện: K-1 phần còn lại được gộp lại để làm tập dữ liệu huấn luyện.
    >Huấn luyện: Mô hình học máy được huấn luyện trên tập huấn luyện (k-1 folds)
    > Đánh giá: Mô hình vừa huấn luyện xong được đánh giá hiệu năng (tính f1-score, accuracy,..) trên tập kiểm định (fold thứ i). Kết quả đánh giá được lưu lại.
  + Tổng hợp kết quả: Sau khi thực hiện K lần lặp thì có K kết quả đánh giá. Hiệu năng cuối cùng của mô hình theo phương pháp CV thường được tính bằng lấy trung bình của K kết quả.
- GridSearchCV: thử tất cả các tổ hợp
- RandomizedSearchCV: thử một số tổ hợp tham số ngẫu nhiên.

B4:Cập nhật kết quả: ghi lại chỉ số hiệu năng của mô hình đã tune vào bảng so sánh chung (file google doc ở trên).


In [5]:
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import GridSearchCV


param_grid = {
    'C': [0.1, 1, 10, 100], # Có thể thêm hoặc bớt giá trị
    'gamma': [0.001, 0.01, 0.1, 1],
    'kernel': ['rbf', 'linear'] # Hoặc thêm 'linear' để# Tập trung vào f1_score của lớp SPAM
}
scorer = make_scorer(f1_score, pos_label=1) # Giả sử 1 là nhãn SPAM nhãn SPAM

# Khởi tạo GridSearchCV với SVC (có thể vẫn giữ class_weight='balanced' hoặc bỏ đi để xem SMOTE có đủ không)
grid_search_smote = GridSearchCV(SVC(class_weight='balanced', random_state=42), 
                                    param_grid, 
                                    cv=5, # Hoặc một giá trị cross-validation khác
                                    scoring=scorer, 
                                    verbose=2, # Để xem tiến trình
                                    n_jobs=-1) # Sử dụng tất cả các core nếu có thể

print("Đang chạy GridSearchCV trên dữ liệu SMOTE...")
grid_search_smote.fit(X_train_smote, y_train_smote)

print("Các tham số SVM tốt nhất sau SMOTE và GridSearchCV:", grid_search_smote.best_params_)
best_svm_after_smote = grid_search_smote.best_estimator_

Đang chạy GridSearchCV trên dữ liệu SMOTE...
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Các tham số SVM tốt nhất sau SMOTE và GridSearchCV: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}


In [6]:

new_messages_for_test = [
    "URGENT! Claim your FREE £1000 prize now! Click http://spamlink.com", # Tin nhắn có vẻ là SPAM
    "Hey, wondering if you're free for coffee later today?", # Tin nhắn có vẻ là HAM
    "Meet singles in your area, text DATE to 88888 T&Cs apply 18+", # Tin nhắn có vẻ là SPAM
    "Remember to buy bread and eggs on your way back.", # Tin nhắn có vẻ là HAM
    "Limited time offer! Exclusive discount just for you!", # Tin nhắn có vẻ là SPAM
    "50 percent discount this Friday only!"
]

print("--- Bắt đầu Test Case ---")
print(f"Số lượng tin nhắn thử nghiệm: {len(new_messages_for_test)}")

# --- 1. Tiền xử lý các tin nhắn mới ---
# Áp dụng hàm tiền xử lý đã có (cần đảm bảo hàm clean_text_v1 đã được định nghĩa/import ở cell trước)
try:
    cleaned_test_messages = [clean_text_v1(msg) for msg in new_messages_for_test]
    print("\nTin nhắn sau khi tiền xử lý:")
    for i, msg in enumerate(cleaned_test_messages):
        print(f"{i+1}: {msg}")
except NameError:
    print("\nLỗi: Hàm 'clean_text_v1' chưa được định nghĩa hoặc import.")
    print("Vui lòng chạy cell định nghĩa/import hàm này trước.")
    # Dừng thực thi cell này nếu hàm chưa có
    raise

# --- 2. Vector hóa các tin nhắn mới ---
# *** Quan trọng: Dùng đối tượng tfidf_vectorizer đã fit từ cell trước, chỉ gọi .transform() ***
try:
    new_messages_tfidf_test = tfidf_vectorizer.transform(cleaned_test_messages)
    print(f"\nĐã vector hóa {len(cleaned_test_messages)} tin nhắn thành ma trận TF-IDF kích thước: {new_messages_tfidf_test.shape}")
except NameError:
    print("\nLỗi: Biến 'tfidf_vectorizer' chưa được định nghĩa.")
    print("Vui lòng chạy cell huấn luyện TF-IDF Vectorizer trước.")
    raise

# --- 3. Dự đoán ---
print("\nĐang thực hiện dự đoán bằng mô hình SVM...")
try:
    # Dự đoán nhãn (0 hoặc 1)
    predictions_test = best_svm_after_smote.predict(new_messages_tfidf_test)
    # Dự đoán xác suất (P(ham), P(spam))
    # Lưu ý: svm_model phải được khởi tạo với probability=True ở cell huấn luyện
    probabilities_test = best_svm_after_smote.predict_proba(new_messages_tfidf_test)
except NameError:
    print("\nLỗi: Biến 'svm_model' chưa được định nghĩa.")
    print("Vui lòng chạy cell huấn luyện mô hình SVM trước.")
    raise
except AttributeError:
     print("\nLỗi: Không thể lấy xác suất.")
     print("Để dùng predict_proba(), mô hình SVM cần được khởi tạo với tham số 'probability=True'.")
     # Gán giá trị mặc định để code không bị lỗi tiếp
     probabilities_test = np.array([[0.5, 0.5]] * len(predictions_test)) # Tạo mảng xác suất giả định

# --- 4. Hiển thị kết quả ---
print("\n--- Kết quả Dự đoán Test Case ---")
label_map = {0: 'ham', 1: 'spam'} # Ánh xạ nhãn số về chữ

for i, original_message in enumerate(new_messages_for_test):
    predicted_label_num = predictions_test[i]
    predicted_label_text = label_map[predicted_label_num]
    prob_ham = probabilities_test[i][0] # Xác suất là ham (lớp 0)
    prob_spam = probabilities_test[i][1] # Xác suất là spam (lớp 1)

    print(f"\nTin nhắn gốc : \"{original_message}\"")
    # print(f"Tin nhắn sạch: \"{cleaned_test_messages[i]}\"") # Bỏ comment nếu muốn xem
    print(f"  -> Dự đoán   : {predicted_label_text.upper()}")
    # Chỉ in xác suất nếu lấy được
    if 'AttributeError' not in locals() or not isinstance(locals()['AttributeError'], AttributeError):
         print(f"  -> Xác suất : [Ham={prob_ham:.4f}, Spam={prob_spam:.4f}]")

print("\n--- Hoàn thành Test Case ---")

--- Bắt đầu Test Case ---
Số lượng tin nhắn thử nghiệm: 6

Tin nhắn sau khi tiền xử lý:
1: urgent claim free 1000 prize click httpspamlinkcom
2: hey wondering youre free coffee later today
3: meet singles area text date 88888 tcs apply 18
4: remember buy bread eggs way back
5: limited time offer exclusive discount
6: 50 percent discount friday

Đã vector hóa 6 tin nhắn thành ma trận TF-IDF kích thước: (6, 5000)

Đang thực hiện dự đoán bằng mô hình SVM...

Lỗi: Không thể lấy xác suất.
Để dùng predict_proba(), mô hình SVM cần được khởi tạo với tham số 'probability=True'.

--- Kết quả Dự đoán Test Case ---

Tin nhắn gốc : "URGENT! Claim your FREE £1000 prize now! Click http://spamlink.com"
  -> Dự đoán   : SPAM
  -> Xác suất : [Ham=0.5000, Spam=0.5000]

Tin nhắn gốc : "Hey, wondering if you're free for coffee later today?"
  -> Dự đoán   : HAM
  -> Xác suất : [Ham=0.5000, Spam=0.5000]

Tin nhắn gốc : "Meet singles in your area, text DATE to 88888 T&Cs apply 18+"
  -> Dự đoán   : SPAM
  ->

In [7]:
print("\nĐánh giá mô hình trên tập kiểm thử (đã SMOTE):")
    # Nếu dùng GridSearchCV, sử dụng best_svm_smote
    # y_pred_smote = best_svm_smote.predict(X_test_tfidf) 
    
    # Nếu huấn luyện trực tiếp
y_pred_smote = best_svm_after_smote.predict(X_test_tfidf)

print("Confusion Matrix (sau SMOTE):")
print(confusion_matrix(y_test, y_pred_smote))
print("\nClassification Report (sau SMOTE):")
print(classification_report(y_test, y_pred_smote, target_names=['HAM (0)', 'SPAM (1)']))


Đánh giá mô hình trên tập kiểm thử (đã SMOTE):
Confusion Matrix (sau SMOTE):
[[945  21]
 [ 17 132]]

Classification Report (sau SMOTE):
              precision    recall  f1-score   support

     HAM (0)       0.98      0.98      0.98       966
    SPAM (1)       0.86      0.89      0.87       149

    accuracy                           0.97      1115
   macro avg       0.92      0.93      0.93      1115
weighted avg       0.97      0.97      0.97      1115



Trong NLP, có rất nhiều cách để vector hoá 1 văn bản text thành ma trận dạng số: Bag_of_words, TF-IDF, word embeddings,..
- Bag_of_words: "I arrive home at 6 o'clock" => [1,1,1,1]
"I come back home and then I go to my grandpa's house" 
- TF-IDF: Term Frequency-Inverse Document Frequency
Term Frequency (Bag_of_words) * Inverse Document Frequency 
IDF = log(number of documents / number of documents in which this word exists)

VD: 'Tôi đi học'
'Tôi đi chơi'
'Mua đồ ăn'

IDF = log(3 / 2) = 0.58 

